In [1]:
import sys
sys.path.append("../")
sys.path.append("../../dcdi")

In [2]:
import numpy as np
import pandas as pd
import torch
import wandb
import matplotlib.pyplot as plt

from sdci.models import SDCI
from sdci.utils import create_intervention_dataset, subset_interventions, train_val_split, compute_metrics, set_random_seed_all
from sdci.third_party.utils.metrics import shd_metric

2023-09-22 17:29:18.068202: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-22 17:29:18.069243: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-22 17:29:18.090398: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-22 17:29:18.090921: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 17:29:18.432086: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

Instructions for updating:
non-resource variables are not supported in the long term


Detecting 1 CUDA device(s).


In [3]:
from dcdi.data import DataManagerFile

# Sweep

In [4]:
dataset_name = "data_p20_e80_n10000_linear_struct"
dataset_i = 1

dmf = DataManagerFile(
    f"../../dcdi/data/perfect/{dataset_name}",
    dataset_i,
    train_samples=1.0, # grab whole dataset
    train=True,
    normalize=True,
    intervention=True,
    intervention_knowledge="known"
)
X = dmf.dataset
masks = dmf.convert_masks(np.arange(X.shape[0]))
n_regimes = (1 - masks).sum(axis=1)
B_true = dmf.adjacency.numpy()

In [5]:
dataset = torch.utils.data.TensorDataset(X, masks, n_regimes)
np.random.seed(2)
train_dataset, val_dataset = train_val_split(dataset, flavor="random", val_fraction=0.2)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

Using cuda:0


In [7]:
def run_sdci(): 
    wandb.init()
    s1_alpha = wandb.config.s1_alpha
    s2_alpha = wandb.config.s2_alpha
    s1_beta = wandb.config.s1_beta
    s2_beta = wandb.config.s2_beta
    max_gamma = wandb.config.max_gamma
    mask_threshold = wandb.config.mask_threshold

    model = SDCI(model_variance_flavor="nn", use_gumbel=True)
    model.train(
        train_dataset,
        val_dataset=val_dataset,
        B_true=B_true,
        device=device,
        verbose=False,
        log_wandb=True,
        wandb_project=f"{dataset_name}",
        wandb_name=f"SDCI_{dataset_i}",
        stage1_kwargs={"n_epochs": 500, "alpha": s1_alpha, "beta": s1_beta, "mask_threshold": mask_threshold},
        stage2_kwargs={"n_epochs": 1500, "alpha": s2_alpha, "beta": s2_beta, "max_gamma": max_gamma, "threshold": 0.5, "freeze_gamma_at_dag": False}
    )

    y_test = B_true.astype(bool)
    y_score = model.get_adjacency_matrix(threshold = True)
    final_shd = shd_metric(y_score, y_test)
    min_dag_threshold = model.compute_min_dag_threshold()
    y_score = model.get_adjacency_matrix(threshold = True)
    final_dag_shd = shd_metric(y_score, y_test)

    wandb.log(dict(final_shd=final_shd, min_dag_threshold=min_dag_threshold, final_dag_shd=final_dag_shd))
    wandb.finish()

In [8]:
sweep_configuration = {
    "method" : "bayes",
    "name": "sdci_new_sim_sweep",
    "metric": {
        "goal": "minimize",
        "name": "final_shd",
    },
    "parameters": {
        "s1_alpha": {"max": 1e-1, "min": 1e-5, "distribution": "log_uniform_values"},
        "s2_alpha": {"max": 1e-4, "min": 1e-7, "distribution": "log_uniform_values"},
        "s1_beta": {"max": 1e-1, "min": 1e-5, "distribution": "log_uniform_values"},
        "s2_beta": {"max": 1e-4, "min": 1e-7, "distribution": "log_uniform_values"},
        "max_gamma": {"max": 1000, "min": 100, "distribution": "log_uniform_values"},
        "mask_threshold": {"max": 0.5, "min": 0.2, "distribution": "uniform"},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 11,
    }
}

In [9]:
wandb.login()

sweep_id = wandb.sweep(sweep=sweep_configuration, project=f"{dataset_name}")
wandb.agent(sweep_id, function=run_sdci)

wandb: Currently logged in as: justinhong (azizi-causal-perturb). Use `wandb login --relogin` to force relogin


Create sweep with ID: x861tvxp
Sweep URL: https://wandb.ai/azizi-causal-perturb/data_p20_e80_n10000_linear_struct/sweeps/x861tvxp


wandb: Agent Starting Run: 6lbxdoc3 with config:
wandb: 	mask_threshold: 0.3691624445843834
wandb: 	max_gamma: 198.35280697917025
wandb: 	s1_alpha: 0.00039654636411213113
wandb: 	s1_beta: 0.0024894272585529007
wandb: 	s2_alpha: 2.2635742006864258e-07
wandb: 	s2_beta: 2.3348680381103637e-05


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'


...

wandb: Agent Starting Run: xjbcpwl8 with config:
wandb: 	mask_threshold: 0.3647678529564011
wandb: 	max_gamma: 213.02711037269543
wandb: 	s1_alpha: 1.8928617387593065e-05
wandb: 	s1_beta: 0.0004403679809318097
wandb: 	s2_alpha: 2.512662910222544e-07
wandb: 	s2_beta: 1.0290348054411669e-05


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'


Epoch 0: loss=26.10, score=310, shd=310.0, gamma=0.00
Epoch 100: loss=4.21, score=310, shd=310.0, gamma=0.00
Epoch 200: loss=3.75, score=310, shd=310.0, gamma=0.00
Epoch 300: loss=3.53, score=310, shd=310.0, gamma=0.00
Epoch 400: loss=3.30, score=310, shd=310.0, gamma=0.00
Recall of mask: 1.0
Fraction of possible edges in mask: 0.95
Epoch 0: loss=27.24, score=310, shd=310.0, gamma=0.00
Epoch 100: loss=59.89, score=45, shd=45.0, gamma=14.21
Epoch 200: loss=29.03, score=70, shd=70.0, gamma=28.42
Epoch 300: loss=27.19, score=70, shd=70.0, gamma=42.63
Epoch 400: loss=26.87, score=70, shd=70.0, gamma=56.85
Epoch 500: loss=26.80, score=70, shd=70.0, gamma=71.06
Epoch 600: loss=26.87, score=70, shd=70.0, gamma=85.27
Epoch 700: loss=26.87, score=70, shd=70.0, gamma=99.48
Epoch 800: loss=26.90, score=70, shd=70.0, gamma=113.69
Epoch 900: loss=26.89, score=70, shd=70.0, gamma=127.90


wandb: Ctrl + C detected. Stopping sweep.


# Run SDCI over all data

In [7]:
def load_dcdi_data(dataset_name, dataset_i):
    dmf = DataManagerFile(
        f"../../dcdi/data/perfect/{dataset_name}",
        dataset_i,
        train_samples=1.0, # grab whole dataset
        train=True,
        normalize=True,
        intervention=True,
        intervention_knowledge="known"
    )
    X = dmf.dataset
    masks = dmf.convert_masks(np.arange(X.shape[0]))
    n_regimes = (1 - masks).sum(axis=1)
    B_true = dmf.adjacency.numpy()
    dataset = torch.utils.data.TensorDataset(X, masks, n_regimes)
    return dataset, B_true

In [10]:
def run_sdci_tuned(dataset, seed=42, device=None):
    set_random_seed_all(seed)
    train_dataset, val_dataset = train_val_split(dataset, flavor="random", val_fraction=0.2)
    stage1_kwargs = {
        "n_epochs": 2000,
        "alpha": 1e-2,
        "beta": 2e-4,
        "mask_threshold": 0.2,
    }
    stage2_kwargs = {
        "learning_rate": 2e-3,
        "n_epochs": 2000,
        "alpha": 5e-5,
        "beta": 5e-5, 
        "max_gamma": 200,
        "freeze_gamma_threshold": 0.3,
        "threshold": 0.3,
    }
    model = SDCI(model_variance_flavor="nn")
    model.train(
        train_dataset,
        val_dataset=val_dataset,
        B_true=B_true,
        device=device,
        verbose=True,
        log_wandb=True,
        wandb_project=f"{dataset_name}",
        wandb_name=f"SDCI_{dataset_i}",
        stage1_kwargs=stage1_kwargs,
        stage2_kwargs=stage2_kwargs,
    )
    y_test = B_true
    fixed_y_score = model.get_adjacency_matrix(threshold = True)
    fixed_metrics_dict = {f"{k}_fixed": v for k,v in compute_metrics(fixed_y_score, y_test).items()}
    min_dag_threshold = model.compute_min_dag_threshold()
    dag_y_score = model.get_adjacency_matrix(threshold = True)
    dag_metrics_dict = {f"{k}_dag": v for k,v in compute_metrics(dag_y_score, y_test).items()}
    dag_metrics_dict["min_dag_threshold"] = min_dag_threshold
    return {**fixed_metrics_dict, **dag_metrics_dict}

In [ ]:
dataset_names = """
data_p20_e20_n10000_nnadd_struct
data_p20_e20_n10000_nn_struct
data_p20_e80_n10000_linear_struct
data_p20_e80_n10000_nnadd_struct
data_p20_e80_n10000_nn_struct
data_p20_e20_n10000_linear_struct
""".split()
# dataset_names = """
# data_p10_e10_n10000_linear_struct
# data_p20_e20_n10000_nnadd_struct
# data_p10_e10_n10000_nnadd_struct
# data_p20_e20_n10000_nn_struct
# data_p10_e10_n10000_nn_struct
# data_p20_e80_n10000_linear_struct
# data_p10_e40_n10000_linear_struct
# data_p20_e80_n10000_nnadd_struct
# data_p10_e40_n10000_nnadd_struct
# data_p20_e80_n10000_nn_struct
# data_p10_e40_n10000_nn_struct
# data_p20_e20_n10000_linear_struct
# """.split()
dataset_is = np.arange(1, 11)

seed = 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

for dataset_name in dataset_names: 
    results_df_rows = []
    for dataset_i in dataset_is:
        dataset, B_true = load_dcdi_data(dataset_name, dataset_i)
    
        np.random.seed(seed)
        res_metrics = run_sdci_tuned(dataset, seed=seed, device=device)
        res_metrics["dataset_i"] = dataset_i
        results_df_rows.append(res_metrics)

        results_df = pd.DataFrame.from_records(results_df_rows)
        results_df.to_csv(f"results/{dataset_name}.csv")

Using cuda:0


alpha,▁▁▁▁▁▁▁▁
dag,▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch_loss,█▂▁▁▁▁▁▁
gamma,▁▁▁▁▁▁▁▁
is_prescreen,▁▁▁▁▁▁▁▁
l1,▁██▇▇▇▇▇
l2,▁▄▅▆▇▇██
n_edges_pred,▁███████
nll,█▁▁▁▁▁▁▁
precision,▁███████


'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'


Epoch 0: loss=27.20, score=22, shd=22.0, gamma=0.00
Node  0: | | 18 4 10 1 11 6 7 16 13 | 12 8 14 9 19 3 15 5 2 17 0
Node  1: | | 4 18 6 0 16 10 11 13 14 7 19 12 | 8 17 2 15 3 9 5 1
Node  2: | | 8 17 | 3 10 9 1 5 11 12 7 18 4 13 19 15 14 0 16 6 2
Node  3: | | | 5 11 6 0 1 16 17 8 12 14 10 18 7 9 4 19 15 2 13 3
Node  4: | | 1 18 0 11 10 7 6 16 13 | 12 8 15 3 19 9 14 2 17 5 4
Node  5: | | | 17 3 2 12 19 13 8 7 1 15 0 6 10 14 11 18 16 4 9 5
Node  6: | | 1 14 16 0 18 13 10 11 19 4 12 7 | 15 8 9 17 2 5 3 6
Node  7: | | 11 10 0 18 4 12 8 1 6 16 | 13 15 14 19 9 3 17 2 5 7
Node  8: | | 2 17 7 11 10 | 0 18 1 4 12 6 16 9 15 19 14 3 13 5 8
Node  9: | | | 10 2 15 0 17 3 14 6 11 16 7 4 1 12 19 18 13 5 8 9
Node 10: | | 0 18 11 7 4 1 6 16 | 13 8 12 9 3 14 2 17 19 5 15 10
Node 11: | | 7 10 18 0 1 4 6 16 12 8 | 13 5 3 9 2 14 15 19 17 11
Node 12: | | 19 14 6 7 | 16 1 11 10 13 0 18 4 8 5 3 2 15 17 9 12
Node 13: | | 15 16 6 1 14 0 18 19 10 12 11 4 | 7 17 8 5 9 3 2 13
Node 14: | | 19 6 16 12 1 13 15 | 0 18

alpha,█████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
dag,▁▁▁▁▁▁▁▁▁▁█▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_loss,█▁▁▁▁▁▁▁▁█▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
gamma,▁▁▁▁▁▁▁▁▁▁▃▆█████████████████
is_prescreen,█████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l1,▃██▇▇▇▇▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l2,▁▄▅▆▆▆▇▇▇▁▂▂▃▄▄▅▅▅▆▆▆▆▇▇▇▇███
n_edges_pred,▁█████▇▇▇▁▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
nll,█▁▁▁▁▁▁▁▁█▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
precision,▁▄▄▄▄▄▄▄▄▁▆▇▇████████████████


...

'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'


Epoch 0: loss=27.42, score=27, shd=27.0, gamma=0.00
Node  0: | | 2 | 6 7 10 17 12 14 3 15 11 18 4 16 13 5 1 8 19 9 0
Node  1: | | 3 19 18 14 5 10 11 4 | 9 17 13 0 6 15 8 7 16 12 2 1
Node  2: | | 0 | 15 4 6 10 1 7 17 18 5 12 14 11 3 8 16 13 19 9 2
Node  3: | | 1 19 5 4 | 9 18 10 13 17 14 15 16 0 8 12 7 2 6 11 3
Node  4: | | 15 19 3 5 1 | 9 18 11 10 2 13 8 7 17 16 6 12 14 0 4
Node  5: | | 19 3 4 1 | 9 18 17 11 16 10 13 14 12 2 8 15 0 6 7 5
Node  6: | | 7 12 | 2 0 10 1 19 9 15 5 13 8 3 18 4 11 17 16 14 6
Node  7: | | 12 6 | 0 4 2 5 18 16 9 15 19 11 8 10 17 13 3 1 14 7
Node  8: | | | 1 4 5 0 18 12 7 11 19 3 17 13 2 9 15 14 6 10 16 8
Node  9: | | 16 11 10 | 19 3 4 12 1 17 18 13 6 2 0 5 15 8 14 7 9
Node 10: | | 11 14 18 16 1 | 9 13 3 19 4 12 5 17 2 7 15 6 8 0 10
Node 11: | | 10 16 18 14 9 1 | 5 15 12 4 3 19 0 17 13 8 7 2 6 11
Node 12: | | 7 6 | 16 0 11 10 4 9 5 8 2 1 18 15 14 17 3 13 19 12
Node 13: | | 10 | 15 3 9 5 16 19 1 11 4 14 18 12 8 6 7 2 0 17 13
Node 14: | | 18 11 10 1 | 17 3 4 7 5 1

alpha,██████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
dag,▁▁▁▁▁▁▁▁▁▁▁█▅▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch_loss,█▁▁▁▁▁▁▁▁▁█▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
gamma,▁▁▁▁▁▁▁▁▁▁▁▃▆█████████████████
is_prescreen,██████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l1,▂███████▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l2,▁▄▄▅▅▆▆▆▇▇▁▂▂▃▄▄▅▅▅▆▆▆▆▇▇▇▇███
n_edges_pred,▁▇████████▁▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
nll,█▁▁▁▁▁▁▁▁▁█▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂
precision,▁▅▄▄▄▄▄▄▄▄▁▇▇█████████████████


'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'


Epoch 0: loss=27.63, score=15, shd=15.0, gamma=0.00
Node  0: | | 15 6 | 2 7 14 4 1 11 19 18 5 13 17 10 9 3 8 12 16 0
Node  1: | | 18 5 10 6 | 8 13 4 17 15 11 16 7 12 9 14 0 3 19 2 1
Node  2: | | | 19 12 9 11 7 10 3 8 1 13 16 4 0 5 17 15 18 14 6 2
Node  3: | | 13 | 15 11 18 12 10 1 6 19 0 2 7 4 5 17 9 16 14 8 3
Node  4: | | | 14 16 11 9 7 19 18 0 8 2 10 1 3 15 6 12 13 5 17 4
Node  5: | | 1 18 6 10 | 8 11 15 7 9 4 2 13 19 3 16 12 0 14 17 5
Node  6: | | 15 18 5 1 0 | 10 11 7 14 3 17 9 19 2 8 12 4 16 13 6
Node  7: | | 11 8 19 | 0 2 5 6 9 16 17 12 15 18 13 10 3 4 14 1 7
Node  8: | | 11 7 19 | 15 5 6 2 17 3 14 4 10 13 18 9 16 1 12 0 8
Node  9: | | | 16 4 13 0 2 6 18 7 1 11 8 15 5 14 10 17 12 19 3 9
Node 10: | | 1 18 5 | 6 0 16 14 3 4 7 15 11 8 12 2 17 9 13 19 10
Node 11: | | 8 7 19 | 0 15 4 14 6 5 9 12 1 17 16 13 2 10 3 18 11
Node 12: | | | 18 16 3 7 6 14 19 5 10 9 1 11 2 8 4 17 13 15 0 12
Node 13: | | 3 | 15 0 7 12 9 11 1 5 17 10 16 18 2 8 14 4 6 19 13
Node 14: | | 16 | 4 15 7 6 9 13 0 5 3 